# Actividad 1

## Requisitos

- Habitación de MxN espacios.
- Número de agentes.
- Porcentaje de celdas inicialmente sucias.
- Tiempo máximo de ejecución.

## Instrucciones

- Inicializa las celdas sucias (ubicaciones aleatorias).
- Todos los agentes empiezan en posiciones aleatorias.
- En cada paso de tiempo:
    - Si la celda está sucia, entonces aspira.
    - Si la celda está limpia, 
        - El agente elije mover a una nueva celda, usando esta selección: norte, este, sur, oeste.
        - El agente se moverá a esa celda siempre y cuando no la haya visitado antes. 
        - Si no tiene opciones para moverse, el agente se moverá a la celda que previamente haya visitado.
        - Si ya no tiene más opciones, el agente se detiene.
- Se ejecuta el tiempo máximo establecido.

Empleando siempre la misma cantidad de agentes (más de 2), determina:

- ¿Cuál es el tiempo necesario hasta que todas las celdas estén limpias?
- Empleando un número máximo de iteraciones, ¿cuál es el porcentaje de celdas limpias después del termino de la simulación?
- En ambos casos, ¿cuál es la cantidad de movimientos realizados por los agentes?

Desarrolla un documento analizando los resultados obtenidos.

## Evaluación 

45% - El código cumple correctamente con la funcionalidad requerida por parte de la actividad.
10% - El código deberá seguir los lineamientos estipulados en el estándar de codificación: liga_estándar_codificación
Download liga_estándar_codificación.
45% - El informe presenta un análisis del problema planteado, así como las conclusiones obtenidas del mismo.

In [ ]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model 

# Necesitamos agentes por celdas, usamos MultiGrid
from mesa.space import MultiGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al azar''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

# Número de iteraciones
MAXVAL = 10000

In [ ]:
class VaccumCleaner(Agent):
    # initialize class, define id and space occupied (model)
    def __init__(self, id, model):
        super().__init__(id, model)
        
        # movement for the agent
        self.options = np.array([
            [-1,-1], [-1,0], [-1, 1],
            [0,-1],          [0, 1],
            [1,-1],  [1,0],  [1,1] ])
        
        # check if you can move around
        def can_move(self,x,y):
            return ((x >= 0 and x <= self.model.grid.width) and
                   (y >= 0 and y <= self.model.grid.height))
        
        
        # define what is done on each step
        def step(self):
            if self.model.cells[[self.pos[0]][self.pos[1]]] == 1:
                self.model.cells[[self.pos[0]][self.pos[1]]] = 0
            else:
                # the agent can collide several times on each step
                i = int(np.random.random() * MAXVAL % len(self.options))
                x_pos = self.pos[0] + self.options[i][0]
                y_pos = self.pos[1] + self.options[i][1]
                if self.can_move(x_pos, y_pos):
                    self.model.grid.move_agent(self, (x_pos, y_pos))

In [ ]:
# save a copy of the matrix
def get_grid(model):
    grid = np.zeros((model.grid.width, model.grid.height))
    for x in range(model.grid.width):
        for y in range(model.grid.height):
            if not model.grid.is_cell_empty((x,y)):
                grid[x][y] = 2
            else:
                grid[x][y] = model.cells[x][y]
    return grid

In [ ]:
class VaccumCleanerModel(Model):
    # initialize class, define space size
    def __init__(self, width, height, num_agents = 1, dirty_cell_percentage = 0.8):
        self.num_agents = num_agents
        self.dirty_cell_percentage = dirty_cell_percentage
        
        # Grid does not wrap-around
        self.grid = MultiGrid(width, height, False)
        
        # Define the activation, generate in each iteration a random model and make a new step
        self.schedule = RandomActivation(self)
        
        # Data collector
        self.datacollector = DataCollector(model_reporters = {"Grid" : get_grid})
        
        # Generate matrix of zeros
        self.cells = np.zeros((width, height))
        amount = int((width*height)*self.dirty_cell_percentage)
        for i in range (amount):
            finished = False
            while not finished:
                x = int(np.random.random() * MAXVAL) % width
                y = int(np.random.random() * MAXVAL) % height
                if self.cells[x][y] == 0:
                    self.cells[x][y] = 1
                    finished = True
        
        # Generate agents in empty cells
        for i in range(num_agents):
            agent = VaccumCleaner(i, self)
            pos = self.grid.find_empty()
            self.grid.place_agent(agent, pos)
            # begin in pos 0,0
            # self.grid.place_agent(agent(0,0))
            # add agent to the schedule
            self.schedule.add(agent)
        
    def all_clean(self):
        return np.all(self.cells == 0)
        
    def step(self):
        # define when snapshot is taken before
        self.datacollector.collect(self)
        self.schedule.step()

In [ ]:
GRID_SIZE = 20
MAX_ITERATIONS = 20

start_time = time.time()
model = VaccumCleanerModel(GRID_SIZE, GRID_SIZE, 1, .9)

i = 0
while i < MAX_ITERATIONS and not model.all_clean():
    model.step()
    i += 1

print(datetime.timedelta(time.time() - start_time))

In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [ ]:
fig, axs = plt.subplots(figsize=(5,5))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.colormaps['viridis'])

def animate(i):
    patch.set_data(all_grid.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames = MAX_ITERATIONS)

In [ ]:
anim